<a href="https://colab.research.google.com/github/Kingzel/pyMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [3]:
from setup import gen_multichoice_features , gen_binary_features , gen_singlechoice_features

In [4]:
evids = pd.read_json("data\\release_evidences.json")

In [5]:
from setup import parse_data
parsed = parse_data(["output\\output_1.csv","output\\output_2.csv","output\\output_3.csv","output\\output_4.csv","output\\output_5.csv"])
train_patients,y = pd.concat(parsed[0::2], ignore_index=True),pd.concat(parsed[1::2],ignore_index=True)

In [6]:
m_collection = gen_multichoice_features(evids)
c_collection =gen_singlechoice_features(evids)
b_collection = gen_binary_features(evids)

In [7]:
all_collections = b_collection+m_collection+c_collection
all_collections.extend(['AGE','SEX'])



In [13]:
from setup import gen_empty
new_df = gen_empty(all_collections,len(train_patients)) 
print(new_df.shape,"\n")
print(new_df.iloc[0:2,:4])

(150000, 914) 

   E_91  E_53  E_159  E_129
0     0     0      0      0
1     0     0      0      0


In [14]:
from setup import populate
new_df = populate(new_df,train_patients)

In [16]:
print(new_df.shape)
print(new_df.iloc[0:2,:4])

(150000, 914)
   E_91  E_53  E_159  E_129
0     1     1      0      0
1     1     1      0      1


In [17]:

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size = 0.35)
print(X_train.head()['AGE'],y_train.head(),sep="\n\n\n")

53400     44
86452      9
15113     49
100985    24
142631    32
Name: AGE, dtype: int64


53400                Tuberculosis
86452                        PSVT
15113     Possible NSTEMI / STEMI
100985           Acute laryngitis
142631           Cluster headache
Name: PATHOLOGY, dtype: object


In [18]:
clf = RandomForestClassifier(n_estimators=10, warm_start=True)
clf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=10, warm_start=True)

In [20]:
y_pred  = clf.predict(X_test)
correct = 0
total =0
for predicted, actual in zip(y_pred,y_test):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,'\n',correct/total,'accuracy')

52158 out of 52500 
 0.9934857142857143 accuracy


In [ ]:
import pickle as p

with open('trained_model.bin', 'wb') as f:
    p.dump(clf, f)


In [ ]:
from setup import parse_data , populate, gen_empty
parsed = parse_data(["data\\release_test_patients\\release_test_patients.csv"])
test_patients = parsed[0]
y=parsed[1]
new_df = gen_empty(all_collections,len(test_patients))
new_df = populate(new_df,test_patients)

In [ ]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf = p.load(f)

In [ ]:
y_pred  = rf.predict(new_df)
correct = 0
total =0
for predicted, actual in zip(y_pred,y):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,correct/total,'accuracy')

DONE